# WFM stitching of 2D ROOT and McStas data

In [ ]:
import os
import uproot
import matplotlib.pyplot as plt
import numpy as np
import dataconfig  # to get paths to data

from dress import wfm

%matplotlib widget

import ipywidgets

# choice of colormap for 2D plots
colormap = plt.cm.get_cmap('gist_earth')

plt.rcParams.update({'figure.max_open_warning': 0})

## McStas

In [ ]:
# McStas 2D file
mcstas_2dfile = "monitor_tx_DENEX.dat"

# check existence of path to folder containing output of McStas simulation
assert os.path.isdir(dataconfig.data_mcstas), \
'The folder which should contain outputs of McStas simulation does not exist.'

path_to_mcstas2D_file = os.path.join(dataconfig.data_mcstas, mcstas_2dfile)

assert os.path.isfile(path_to_mcstas2D_file), \
'There is an issue with the chosen McStas 2D datafile'

### Extract shape of output data

These values will be used to determine how many lines to read one array

In [ ]:
with open(path_to_mcstas2D_file, 'r') as file:
    for line in file:
        if "array_2d" in line:
            type_array = line.rstrip()
            start = type_array.find('(') + 1
            end = type_array.find(')', start)
            nx_value, ny_value = map(int, type_array[start:end].split(','))
        if "xylimits" in line:
            xylims = np.array(line.split(':')[1].split()).astype(float)

print(f'Limits of x- and y-axis: {xylims}\nNumber of points: nx = {nx_value}, ny = {ny_value}')

### Read and plot data

In [ ]:
data2d = np.genfromtxt(path_to_mcstas2D_file, max_rows=ny_value)

#flip data along y axis 
data2d_mcstas = np.flip(data2d, 0)

# define x, y axes (bin-centered)
dx = (xylims[1] - xylims[0]) / float(nx_value)
dy = (xylims[3] - xylims[2]) / float(ny_value)
xaxis_mcstas = np.linspace(xylims[0] + 0.5*dx, xylims[1] - 0.5*dx, nx_value) * 1.0e6
yaxis_mcstas = np.linspace(xylims[2] + 0.5*dx, xylims[3] - 0.5*dy, nx_value)

In [ ]:
fig1, ax1 = plt.subplots()
cont1 = ax1.contourf(xaxis_mcstas, yaxis_mcstas, data2d_mcstas, cmap=colormap)    
ax1.set_title(f"Before stitching: {mcstas_2dfile}")
cbar1 = fig1.colorbar(cont1)

### 1D slices of initial data

In [ ]:
# function to be updated when changing the value to calculate the projection
def plots_for_projection_mcstas(index_selected):
    """ Plot 2D and 1D projection for user-specified range of y-values"""
    CS = ax1_1d[0].contourf(xaxis_mcstas, yaxis_mcstas, data2d_mcstas, cmap=colormap)
    ax1_1d[0].hlines(yaxis_mcstas[index_selected], min(xaxis_mcstas), max(xaxis_mcstas), 'orange')
    
    # calculate projection
    ax1_1d[1].grid()
    ax1_1d[1].set_xlim(min(xaxis_mcstas), max(xaxis_mcstas))
    ax1_1d[1].set_ylim(np.min(data2d_mcstas), np.max(data2d_mcstas))
    
    line_mcstas, = ax1_1d[1].plot(xaxis_mcstas, data2d_mcstas[index_selected,:])
    fig1_1d.colorbar(CS, ax=[ax1_1d[0]], location='right')
    fig1_1d.canvas.draw()
    
style_textbox = {'description_width': 'initial'}

def update_1dplot(change):
    ax1_1d[0].clear()
    ax1_1d[1].clear() 
    plots_for_projection_mcstas(change.new)

set_selection = ipywidgets.IntSlider(
    value=0,
    min=0,
    max=len(yaxis_mcstas)-1,
    step=1,
    description='Index:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout_format='d',
    style=style_textbox,
    layout = ipywidgets.Layout(width='400px')
)

fig1_1d, ax1_1d = plt.subplots(2, 1, constrained_layout=True, sharex=True)
fig1_1d.canvas.header_visible = False
plots_for_projection_mcstas(0)

set_selection.observe(update_1dplot, 'value')

play = ipywidgets.Play(
    value=0,
    min=0,
    max=len(yaxis_mcstas)-1,
    step=1,
    interval=200,
    description="Press play",
    disabled=False
)

ipywidgets.jslink((play, 'value'), (set_selection, 'value'))
ipywidgets.HBox([play, set_selection])

### Determine position and amplitude of last peak to be compared to those from the stitched data

In [ ]:
# Index where x > 52000
idx = (np.abs(xaxis_mcstas - 52000)).argmin()
last_max = []
pos_last_max = []
for i in range(len(yaxis_mcstas)):
    idx_max = np.argmax(data2d_mcstas[i, idx:])
    last_max.append(data2d_mcstas[i, idx+idx_max])
    pos_last_max.append(xaxis_mcstas[idx+idx_max])

### Determine position and amplitude of next to last peak to be compared to those from the stitched data

In [ ]:
idxM  = (np.abs(xaxis_mcstas-52000)).argmin()
def xminb(i):
    return 43500 + (45500-43500)*(i)/len(yaxis_mcstas)

nlast_max = []
pos_nlast_max = []

for i in range(len(yaxis_mcstas)):
    idxm  = (np.abs(xaxis_mcstas-xminb(i))).argmin()
    idx_max = np.argmax(data2d_mcstas[i, idxm:idxM])
    nlast_max.append(data2d_mcstas[i, idxm+idx_max])
    pos_nlast_max.append(xaxis_mcstas[idxm+idx_max])

### Check determination of last 2 max

In [ ]:
# 1D slices of initial data & check determination of last max 
# function to be updated when changing the value to calculate the projection
def plots_for_projection_mcstas0(index_selected):
    """ Plot 2D and 1D projection for user-specified range of y-values"""
    CS = ax1_1d0[0].contourf(xaxis_mcstas, yaxis_mcstas, data2d_mcstas, cmap=colormap)
    ax1_1d0[0].hlines(yaxis_mcstas[index_selected], min(xaxis_mcstas), max(xaxis_mcstas), 'orange')
    ax1_1d0[0].grid()
    # calculate projection
    ax1_1d0[1].grid()
    ax1_1d0[1].set_xlim(min(xaxis_mcstas), max(xaxis_mcstas))
    ax1_1d0[1].set_ylim(np.min(data2d_mcstas), np.max(data2d_mcstas))
    
    line_mcstas, = ax1_1d0[1].plot(xaxis_mcstas, data2d_mcstas[index_selected,:])
    line_mcstas, = ax1_1d0[1].plot(pos_last_max, last_max)
    line_mcstas, = ax1_1d0[1].plot(pos_nlast_max, nlast_max)
    
    fig1_1d0.colorbar(CS, ax=[ax1_1d0[0]], location='right')
    fig1_1d0.canvas.draw()
    
style_textbox = {'description_width': 'initial'}

def update_1dplot0(change):
    ax1_1d0[0].clear()
    ax1_1d0[1].clear() 
    plots_for_projection_mcstas0(change.new)

set_selection0 = ipywidgets.IntSlider(
    value=0, min=0, max=len(yaxis_mcstas)-1, step=1,
    description='Index:',
    continuous_update=False,
    orientation='horizontal',
    readout_format='d',
    style=style_textbox,
    layout = ipywidgets.Layout(width='400px'))

fig1_1d0, ax1_1d0 = plt.subplots(2, 1, constrained_layout=True, sharex=True)
fig1_1d0.canvas.header_visible = False
plots_for_projection_mcstas0(0)

set_selection0.observe(update_1dplot0, 'value')
set_selection0

###  Stitching

In [ ]:
# Get the V20 instrument chopper setup
v20setup = wfm.v20.setup()

# Modification of settings
position_source_chopper1 = 21.729
position_source_chopper2 = 21.759
avg_pos_source_choppers = 0.5 * (position_source_chopper1 + position_source_chopper2)
print(f"Average position of the source choppers (m): {avg_pos_source_choppers:.4f}")

# WFM1
# opening of the slits in degrees
mcstas_wfm1_theta0 = [10.9872, 15.2964, 19.3032, 23.0076, 26.46, 29.6856]
# offset to the positions of the slits
mcstas_wfm1_offset = 14.8428
# centres of the slits in degrees
mcstas_wfm1_mid_slits = [89.208, 148.1382, 202.9104, 253.827, 301.14, 345.1392]

v20setup['choppers']['WFM1'].openings = np.deg2rad([
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[0] - .5 * mcstas_wfm1_theta0[0],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[0] + .5 * mcstas_wfm1_theta0[0],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[1] - .5 * mcstas_wfm1_theta0[1],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[1] + .5 * mcstas_wfm1_theta0[1],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[2] - .5 * mcstas_wfm1_theta0[2],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[2] + .5 * mcstas_wfm1_theta0[2],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[3] - .5 * mcstas_wfm1_theta0[3],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[3] + .5 * mcstas_wfm1_theta0[3],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[4] - .5 * mcstas_wfm1_theta0[4],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[4] + .5 * mcstas_wfm1_theta0[4],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[5] - .5 * mcstas_wfm1_theta0[5],
    mcstas_wfm1_offset + mcstas_wfm1_mid_slits[5] + .5 * mcstas_wfm1_theta0[5]
])

v20setup['choppers']['WFM1'].phase = np.deg2rad(17.1)

v20setup['choppers']['WFM1'].distance = 28.594 - 0.292 * 0.5 - avg_pos_source_choppers

# WFM2
# opening of the slits in degrees
mcstas_wfm2_theta0 = [10.9872, 15.2964, 19.3032, 19.3032, 23.0076, 29.6856]
# offset to the positions of the slits
mcstas_wfm2_offset = 14.8428
# centres of the slits in degrees
mcstas_wfm2_mid_slits = [70.5348, 133.749, 192.528, 245.322, 296.2386, 345.1644]

v20setup['choppers']['WFM2'].openings = np.deg2rad([
    mcstas_wfm2_mid_slits[0] + mcstas_wfm2_offset - .5 * mcstas_wfm2_theta0[0],
    mcstas_wfm2_mid_slits[0] + mcstas_wfm2_offset + .5 * mcstas_wfm2_theta0[0],
    mcstas_wfm2_mid_slits[1] + mcstas_wfm2_offset - .5 * mcstas_wfm2_theta0[1],
    mcstas_wfm2_mid_slits[1] + mcstas_wfm2_offset + .5 * mcstas_wfm2_theta0[1],
    mcstas_wfm2_mid_slits[2] + mcstas_wfm2_offset - .5 * mcstas_wfm2_theta0[2],
    mcstas_wfm2_mid_slits[2] + mcstas_wfm2_offset + .5 * mcstas_wfm2_theta0[2],
    mcstas_wfm2_mid_slits[3] + mcstas_wfm2_offset - .5 * mcstas_wfm2_theta0[3],
    mcstas_wfm2_mid_slits[3] + mcstas_wfm2_offset + .5 * mcstas_wfm2_theta0[3],
    mcstas_wfm2_mid_slits[4] + mcstas_wfm2_offset - .5 * mcstas_wfm2_theta0[4],
    mcstas_wfm2_mid_slits[4] + mcstas_wfm2_offset + .5 * mcstas_wfm2_theta0[4],
    mcstas_wfm2_mid_slits[5] + mcstas_wfm2_offset - .5 * mcstas_wfm2_theta0[5],
    mcstas_wfm2_mid_slits[5] + mcstas_wfm2_offset + .5 * mcstas_wfm2_theta0[5]])

v20setup['choppers']['WFM2'].phase = np.deg2rad(46.76)

v20setup['choppers']['WFM2'].distance = 28.594 + 0.292 * 0.5 - avg_pos_source_choppers

# FOC1
# opening of the slits in degrees
mcstas_fol1_theta0 = [20.64, 23.24, 21.81, 17.87, 15.76, 24.47]
# offset to the positions of the slits
mcstas_fol1_offset = 12.235
# centres of the slits in degrees
mcstas_fol1_mid_slits = [74.67, 136.67, 194.315, 245.335, 294.92, 347.765]

v20setup['choppers']['FOL1'].openings = np.deg2rad([
    mcstas_fol1_mid_slits[0] + mcstas_fol1_offset - .5 * mcstas_fol1_theta0[0],
    mcstas_fol1_mid_slits[0] + mcstas_fol1_offset + .5 * mcstas_fol1_theta0[0],
    mcstas_fol1_mid_slits[1] + mcstas_fol1_offset - .5 * mcstas_fol1_theta0[1],
    mcstas_fol1_mid_slits[1] + mcstas_fol1_offset + .5 * mcstas_fol1_theta0[1],
    mcstas_fol1_mid_slits[2] + mcstas_fol1_offset - .5 * mcstas_fol1_theta0[2],
    mcstas_fol1_mid_slits[2] + mcstas_fol1_offset + .5 * mcstas_fol1_theta0[2],
    mcstas_fol1_mid_slits[3] + mcstas_fol1_offset - .5 * mcstas_fol1_theta0[3],
    mcstas_fol1_mid_slits[3] + mcstas_fol1_offset + .5 * mcstas_fol1_theta0[3],
    mcstas_fol1_mid_slits[4] + mcstas_fol1_offset - .5 * mcstas_fol1_theta0[4],
    mcstas_fol1_mid_slits[4] + mcstas_fol1_offset + .5 * mcstas_fol1_theta0[4],
    mcstas_fol1_mid_slits[5] + mcstas_fol1_offset - .5 * mcstas_fol1_theta0[5],
    mcstas_fol1_mid_slits[5] + mcstas_fol1_offset + .5 * mcstas_fol1_theta0[5]])

v20setup['choppers']['FOL1'].phase = np.deg2rad(32.4)

v20setup['choppers']['FOL1'].distance = 30.444 - avg_pos_source_choppers

# FOC2
# opening of the slits in degrees
mcstas_fol2_theta0 = [36.6, 36.06, 30.21, 26.88, 24.56, 29.11]
# offset to the positions of the slits
mcstas_fol2_offset = 14.555
# centres of the slits in degrees
mcstas_fol2_mid_slits = [98.06, 154.44, 206.835, 254.25, 299.41, 345.445]

v20setup['choppers']['FOL2'].openings = np.deg2rad([
    mcstas_fol2_mid_slits[0] + mcstas_fol2_offset - .5 * mcstas_fol2_theta0[0],
    mcstas_fol2_mid_slits[0] + mcstas_fol2_offset + .5 * mcstas_fol2_theta0[0],
    mcstas_fol2_mid_slits[1] + mcstas_fol2_offset - .5 * mcstas_fol2_theta0[1],
    mcstas_fol2_mid_slits[1] + mcstas_fol2_offset + .5 * mcstas_fol2_theta0[1],
    mcstas_fol2_mid_slits[2] + mcstas_fol2_offset - .5 * mcstas_fol2_theta0[2],
    mcstas_fol2_mid_slits[2] + mcstas_fol2_offset + .5 * mcstas_fol2_theta0[2],
    mcstas_fol2_mid_slits[3] + mcstas_fol2_offset - .5 * mcstas_fol2_theta0[3],
    mcstas_fol2_mid_slits[3] + mcstas_fol2_offset + .5 * mcstas_fol2_theta0[3],
    mcstas_fol2_mid_slits[4] + mcstas_fol2_offset - .5 * mcstas_fol2_theta0[4],
    mcstas_fol2_mid_slits[4] + mcstas_fol2_offset + .5 * mcstas_fol2_theta0[4],
    mcstas_fol2_mid_slits[5] + mcstas_fol2_offset - .5 * mcstas_fol2_theta0[5],
    mcstas_fol2_mid_slits[5] + mcstas_fol2_offset + .5 * mcstas_fol2_theta0[5]])

v20setup['choppers']['FOL2'].phase = np.deg2rad(342.27-360)

v20setup['choppers']['FOL2'].distance = 37.544 - avg_pos_source_choppers

# Detector's position
# The DENEX detector is rotated by 90 degrees

v20setup['info']['detector_positions']['DENEX'] = 50.55 + 0.945 - avg_pos_source_choppers 
v20setup['info']["wfm_choppers_midpoint"] = 0.5 * (v20setup['choppers']['WFM1'].distance + v20setup['choppers']['WFM2'].distance)

print(f"Position of DENEX (m): {v20setup['info']['detector_positions']['DENEX']:.4f}, position of middle of WFM1 and WFM2 (m): {v20setup['info']['wfm_choppers_midpoint']:.4f}")

In [ ]:
# check new settings
print(v20setup.keys())
print('==============================')
for key in v20setup['info'].keys():
    print(key, v20setup['info'][key])
print('==============================')
for key in v20setup['choppers'].keys():
    print(f"{key}:")
    print(f"frequency {v20setup['choppers'][key].frequency}")
    print(f"openings {v20setup['choppers'][key].openings}")
    print(f"omega {v20setup['choppers'][key].omega}")
    print(f"distance {v20setup['choppers'][key].distance}")
    print(f"phase {v20setup['choppers'][key].phase}")
    print(f"name {v20setup['choppers'][key].name}\n")

#### Perform stitching

In [ ]:
# Get WFM frame parameters and generate TOF diagram
frames = wfm.get_frames(instrument=v20setup, plot=True)

In [ ]:
# Plot frame locations onto raw data
fig2, ax2 = plt.subplots()
ax2.plot(xaxis_mcstas, np.sum(data2d_mcstas, axis=0))
ax2.set_title('McStas - frame boundaries for stitching')

for gap in frames['DENEX']["gaps"]:
    ax2.axvline(x=gap, linestyle='--', color='r')
    
ax2.grid()

In [ ]:
# Stitch the 2d histogram
mcstas_stitched = wfm.stitch(x=xaxis_mcstas, y=data2d_mcstas, frames=frames['DENEX'])

In [ ]:
fig3, ax3 = plt.subplots(2, 1, constrained_layout=True, sharex=True)
cont3 = ax3[0].contourf(xaxis_mcstas, yaxis_mcstas, mcstas_stitched, cmap=colormap)
cbar3 = fig3.colorbar(cont3, ax=[ax3[0]], location='right')
ax3[0].set_title('McStas - Stitched data')
ax3[1].plot(xaxis_mcstas, np.sum(mcstas_stitched, axis=0))
ax3[1].grid()

In [ ]:
# 1D slices of initial data
# function to be updated when changing the value to calculate the projection
def plots_for_projection_stitched(index_selected):
    """ Plot 2D and 1D projection for user-specified range of y-values"""
    CS = ax_stitched1d[0].contourf(xaxis_mcstas, yaxis_mcstas, mcstas_stitched, cmap=colormap)
    ax_stitched1d[0].hlines(yaxis_mcstas[index_selected], min(xaxis_mcstas), max(xaxis_mcstas), 'orange')
    
    # calculate projection
    ax_stitched1d[1].grid()
    ax_stitched1d[1].set_xlim(min(xaxis_mcstas), max(xaxis_mcstas))
    ax_stitched1d[1].set_ylim(np.min(mcstas_stitched), np.max(mcstas_stitched))
    
    line_mcstas, = ax_stitched1d[1].plot(xaxis_mcstas, mcstas_stitched[index_selected,:])
    fig_stitched1d.colorbar(CS, ax=[ax_stitched1d[0]], location='right')
    fig_stitched1d.canvas.draw()
    
style_textbox = {'description_width': 'initial'}

def update_1dstitched(change):
    ax_stitched1d[0].clear()
    ax_stitched1d[1].clear() 
    plots_for_projection_stitched(change.new)

set_selection_stitched = ipywidgets.IntSlider(
    value=0,
    min=0,
    max=len(yaxis_mcstas)-1,
    step=1,
    description='Index:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout_format='d',
    style=style_textbox,
    layout = ipywidgets.Layout(width='400px')
)

fig_stitched1d, ax_stitched1d = plt.subplots(2, 1, constrained_layout=True, sharex=True)
fig_stitched1d.canvas.header_visible = False
plots_for_projection_stitched(0)

set_selection_stitched.observe(update_1dstitched, 'value')

play_stitched = ipywidgets.Play(
    value=0,
    min=0,
    max=len(yaxis_mcstas)-1,
    step=1,
    interval=200,
    description="Press play",
    disabled=False
)

ipywidgets.jslink((play_stitched, 'value'), (set_selection_stitched, 'value'))
ipywidgets.HBox([play_stitched, set_selection_stitched])

#### Determine position and amplitude of last peak

In [ ]:
idx = (np.abs(xaxis_mcstas-40000)).argmin()
last_max_stitched = []
pos_last_max_stitched = []
for i in range(len(yaxis_mcstas)):
    idx_max_stitched = np.argmax(mcstas_stitched[i, idx:])
    last_max_stitched.append(mcstas_stitched[i, idx+idx_max_stitched]) 
    pos_last_max_stitched.append(xaxis_mcstas[idx+idx_max_stitched])

#### Determine position and amplitude of next to last peak

In [ ]:
def xminb(i):
    return 32500 + (33500-32500)*(i)/len(yaxis_mcstas)

# find next to last max
idxM  = (np.abs(xaxis_mcstas-39000)).argmin()

nlast_max_stitched = []
pos_nlast_max_stitched = []

for i in range(len(yaxis_mcstas)):
    idxm  = (np.abs(xaxis_mcstas-xminb(i))).argmin()
    idx_max = np.argmax(mcstas_stitched[i, idxm:idxM])
    nlast_max_stitched.append(mcstas_stitched[i, idxm+idx_max])
    pos_nlast_max_stitched.append(xaxis_mcstas[idxm+idx_max])

In [ ]:
# # fitting position of peaks
# from scipy import optimize

# # define linear fitting model

# def test_func(x, a, b):
#     return a + b * x

# # initial - fit only part before jump
# params_ini, params_cov_ini = optimize.curve_fit(test_func, np.arange(266), 
#                                                 pos_last_max[:266],
#                                                 p0=[2, 2])
# # stitched
# params_stitched, params_cov_stitched = optimize.curve_fit(test_func, 
#                                                           np.arange(len(pos_last_max_stitched)),
#                                                           pos_last_max_stitched,
#                                                           p0=[2, 2])

# print(f"Fitting results for ini: {params_ini} and stitched {params_stitched}")

#### Check determination of last two max for stitched data

In [ ]:
# 1D slices of initial data
# function to be updated when changing the value to calculate the projection
def plots_for_projection_stitched(index_selected):
    """ Plot 2D and 1D projection for user-specified range of y-values"""
    CS = ax_stitched1d[0].contourf(xaxis_mcstas, yaxis_mcstas, mcstas_stitched, cmap=colormap)
    ax_stitched1d[0].hlines(yaxis_mcstas[index_selected], min(xaxis_mcstas), max(xaxis_mcstas), 'orange')
    ax_stitched1d[0].grid()
    # calculate projection
    ax_stitched1d[1].grid()
    ax_stitched1d[1].set_xlim(min(xaxis_mcstas), max(xaxis_mcstas))
    ax_stitched1d[1].set_ylim(np.min(mcstas_stitched), np.max(mcstas_stitched))
    
    line_mcstas, = ax_stitched1d[1].plot(xaxis_mcstas, mcstas_stitched[index_selected,:])
    # line_mcstasini, = ax_stitched1d[1].plot(xaxis_mcstas, data2d_mcstas[index_selected,:])
    
    ax_stitched1d[1].plot(pos_last_max_stitched, last_max_stitched)
    ax_stitched1d[1].plot(pos_nlast_max_stitched, nlast_max_stitched)
    # ax_stitched1d[1].plot(pos_last_max, last_max) 
    # ax_stitched1d[1].plot(pos_nlast_max, nlast_max) 
    
    fig_stitched1d.colorbar(CS, ax=[ax_stitched1d[0]], location='right')
    fig_stitched1d.canvas.draw()
    
style_textbox = {'description_width': 'initial'}

def update_1dstitched(change):
    ax_stitched1d[0].clear()
    ax_stitched1d[1].clear() 
    plots_for_projection_stitched(change.new)

set_selection_stitched = ipywidgets.IntSlider(
    value=0,
    min=0, max=len(yaxis_mcstas)-1, step=1,
    description='Index:',
    continuous_update=False,
    orientation='horizontal',
    readout_format='d',
    style=style_textbox,
    layout = ipywidgets.Layout(width='400px'))

fig_stitched1d, ax_stitched1d = plt.subplots(2, 1, constrained_layout=True, sharex=True)
fig_stitched1d.canvas.header_visible = False
plots_for_projection_stitched(0)

set_selection_stitched.observe(update_1dstitched, 'value')
set_selection_stitched

### Comparison of amplitudes and positions of last two peaks for initial and stitched data

In [ ]:
# plot amplitude and position of last peak for initial and stitched McStas data
fig_max, ax_max = plt.subplots(2, 1, sharex=True)
fig_max.suptitle('Amplitude and position of last peak for initial and stitched McStas data')
ax_max[0].plot(last_max, label='ini')
ax_max[0].plot(last_max_stitched, label='stitched')
ax_max[0].set_ylabel('amplitude of last peak')
ax_max[0].grid()
ax_max[0].legend()

ax_max[1].plot(pos_last_max, label='ini')
ax_max[1].plot(pos_last_max_stitched, label='stitched')
ax_max[1].set_ylabel('position of last peak')

#ax_max[1].plot(params_ini[0]+ params_ini[1]*np.arange(266), 'r--', label='fit 1st part ini')
#ax_max[1].plot(params_stitched[0]+ params_stitched[1]*np.arange(len(pos_last_max_stitched)), 'b--', label='fit stitched')
ax_max[1].grid()
ax_max[1].legend(loc=(0.7, 0.325)) 

ax_max[1].set_xlabel('index for 1d slice');

In [ ]:
fig_max0, ax_max0 = plt.subplots(2, 1, sharex=True)
ax_max0[0].plot(nlast_max, label='ini')
ax_max0[0].plot(nlast_max_stitched, label='stitched')
ax_max0[0].set_ylabel('amplitude of next to last peak')
ax_max0[0].grid()
fig_max0.suptitle('Amplitude and position of next to last peak for initial & stitched McStas data')

ax_max0[0].legend()

ax_max0[1].plot(pos_nlast_max, label='ini')
ax_max0[1].plot(pos_nlast_max_stitched, label='stitched')
ax_max0[1].set_ylabel('position of next to last peak')
ax_max0[1].hlines(frames['DENEX']["gaps"][-2], 0, len(pos_last_max_stitched), label='gap')
ax_max0[1].grid()
ax_max0[1].legend()

ax_max0[1].set_xlabel('index');

In [ ]:
fig4, ax4 = plt.subplots(1, 2, constrained_layout=True)
fig4.suptitle('McStas')
ax4[0].contourf(xaxis_mcstas, yaxis_mcstas, data2d_mcstas, cmap=colormap)
ax4[0].set_title('Raw data')

ax4[1].set_title('Stitched data')
ax4[1].contourf(xaxis_mcstas, yaxis_mcstas, mcstas_stitched, cmap=colormap);

## ROOT file

In [ ]:
# ROOT file
assert os.path.isdir(dataconfig.data_root), \
    'The path to the folder which should contain ROOT files does not exist.'

ROOT_file_sp3 = "Spectrum03_DENEX006_1_18-02-05_0000.root"

path_to_root_file = os.path.join(dataconfig.data_root, ROOT_file_sp3)

# Some metadata related to TOF channel for ROOT file
tof_tick = 25e-3 # in microseconds (25 ns)

In [ ]:
# open a ROOT file and extract only one 2D dataset specified in selected_dataset
# Note the vertical axis of 2D datasets is inverted

key_spectrum ='Spectrum03'
dir_with_data = 'Meas_3'
selected_dataset = 'H_TOF,X1-X2_User_2D2_dsp_after_run_3'

with uproot.open(path_to_root_file)[dir_with_data] as myFile:

    for key in myFile.keys():
        if 'BoardParam_run' in str(key):
            myObject = myFile[key]
            nb_xbins = int(myObject.member('fEntries'))
                        
            for i in range(nb_xbins):
                if 'TOF_Time_Channel_Width' in myObject.axis(axis=0).labels()[i]:
                    TOF_Time_Channel_Width = myObject.counts(False)[i]
                elif 'TOF_Window_Delay_Register' in myObject.axis(axis=0).labels()[i]:
                    TOF_Window_Delay_Register = myObject.counts(False)[i]
                    
                # print(f"{myObject.axis(axis=0).labels()[i]}: {myObject.counts(False)[i]}")

        # 2D contourplot
        if selected_dataset in str(key):
            myObject = myFile[key]
            data2d_root_sp3 =  np.flip(myObject.counts(False), 1).transpose()
            # create x- and y-axis
            xaxis = myObject.axis(axis=0).edges()[:-1]
            yaxis = myObject.axis(axis=1).edges()[:-1]

In [ ]:
# Convert TOF channels to microseconds:
# a TOF channel width is 2773 ticks and one tick is 25ns.
# Then also add a 6.25ms delay

xaxis = (xaxis * TOF_Time_Channel_Width + TOF_Window_Delay_Register) * tof_tick

In [ ]:
fig5, ax5 = plt.subplots()
cont5 = ax5.contourf(xaxis, yaxis, data2d_root_sp3, cmap=colormap)
ax5.set_title(ROOT_file_sp3)
cbar5 = fig5.colorbar(cont5)

### Stitching

In [ ]:
# Get the V20 instrument chopper setup
v20setup = wfm.v20.setup()

#### Modifications of settings

</div> <div class="alert alert-block alert-danger"><b>TO DO: check parameters for experimental setup: choppers, detector's distance... </b>

In [ ]:
position_source_chopper1 = 21.89
position_source_chopper2 = 21.91
avg_pos_source_choppers = 0.5 * (position_source_chopper1 + position_source_chopper2)
print(f"Average position of the source choppers (m): {avg_pos_source_choppers}")

# WFM1
v20setup['choppers']['WFM1'].phase = np.deg2rad(17.1)

v20setup['choppers']['WFM1'].distance = 28.57 - avg_pos_source_choppers

# WFM2
v20setup['choppers']['WFM2'].phase = np.deg2rad(46.76)

v20setup['choppers']['WFM2'].distance = 28.91 - avg_pos_source_choppers

# FOC1
v20setup['choppers']['FOL1'].phase = np.deg2rad(32.4)

v20setup['choppers']['FOL1'].distance = 30.5 - avg_pos_source_choppers

# FOC2
v20setup['choppers']['FOL2'].phase = np.deg2rad(342.27-360)

v20setup['choppers']['FOL2'].distance = 37.6 - avg_pos_source_choppers

# Change distance of detector
v20setup['info']['detector_positions']['DENEX'] = 50.6 + 0.945 - avg_pos_source_choppers 

v20setup['info']["wfm_choppers_midpoint"] = 0.5 * (v20setup['choppers']['WFM1'].distance + v20setup['choppers']['WFM2'].distance)

print(f"{v20setup['info']['detector_positions']['DENEX']}, {v20setup['info']['wfm_choppers_midpoint']}")

In [ ]:
print(v20setup.keys())
print('==============================')
for key in v20setup['info'].keys():
    print(key, v20setup['info'][key])
print('==============================')
for key in v20setup['choppers'].keys():
    print(f"{key}:")
    print(f"frequency {v20setup['choppers'][key].frequency}")
    print(f"openings {v20setup['choppers'][key].openings}")
    print(f"omega {v20setup['choppers'][key].omega}")
    print(f"distance {v20setup['choppers'][key].distance}")
    print(f"phase {v20setup['choppers'][key].phase}")
    print(f"name {v20setup['choppers'][key].name}\n")

#### Perform stitching

In [ ]:
# Get WFM frame parameters and generate TOF diagram
frames = wfm.get_frames(instrument=v20setup, plot=True)

In [ ]:
# Plot frame locations onto raw data
fig6, ax6 = plt.subplots()
ax6.plot(xaxis, np.sum(data2d_root_sp3, axis=0))
ax6.grid()
ax6.set_title('ROOT - frame boundaries for stitching')
for gap in frames['DENEX']["gaps"]:
    ax6.axvline(x=gap, linestyle='--', color='r');

In [ ]:
# Stitch the 2d histogram
root_stitched_sp3 = wfm.stitch(x=xaxis, y=data2d_root_sp3, frames=frames['DENEX'])

In [ ]:
fig7, ax7 = plt.subplots(2, 1, constrained_layout=True, sharex=True)
ax7[0].set_title('Final stitched plot - ROOT Spectrum 3')
cont7 = ax7[0].contourf(xaxis, yaxis, root_stitched_sp3, cmap=colormap)
fig7.colorbar(cont7, ax=[ax7[0]], location='right')

ax7[1].plot(xaxis, np.sum(root_stitched_sp3, axis=0))
ax7[1].grid()

In [ ]:
fig8, ax8 = plt.subplots(1, 2, constrained_layout=True)
fig8.suptitle('ROOT Spectrum 3')
ax8[0].contourf(xaxis, yaxis, data2d_root_sp3, cmap=colormap)
ax8[0].set_title('Raw data')

ax8[1].set_title('Stitched data')
ax8[1].contourf(xaxis, yaxis, root_stitched_sp3, cmap=colormap);

</div> <div class="alert alert-block alert-danger"><b>TO DO: settings for stitching to be updated. Below the old values are still used. </b>

## Spectrum 11

In [ ]:
# open a ROOT file and extract only one 2D dataset specified in selected_dataset
# Note the vertical axis of 2D datasets is inverted

ROOT_file_sp11 = "Spectrum11_DENEX006_1_18-02-09_0001.root"
path_to_root_file = os.path.join(dataconfig.data_root, ROOT_file_sp11)
key_spectrum ='Spectrum11'
dir_with_data = 'Meas_1'
selected_dataset = 'H_TOF,X1-X2_User_2D4_dsp_after_run_1'

with uproot.open(path_to_root_file)[dir_with_data] as myFile:

    for key in myFile.keys():
        if 'BoardParam_run' in str(key):
            myObject = myFile[key]
            nb_xbins = int(myObject.member('fEntries'))
                        
            for i in range(nb_xbins):
                if 'TOF_Time_Channel_Width' in myObject.axis(axis=0).labels()[i]:
                    TOF_Time_Channel_Width = myObject.counts(False)[i]
                elif 'TOF_Window_Delay_Register' in myObject.axis(axis=0).labels()[i]:
                    TOF_Window_Delay_Register = myObject.counts(False)[i]  
                # print(f"{myObject.axis(axis=0).labels()[i]}: {myObject.counts(False)[i]}")

        # 2D contourplot
        if selected_dataset in str(key):
            data2d_root_sp11 =  np.flip(myFile[key].counts(False), 1).transpose()
            # create x- and y-axis
            xaxis = myFile[key].axis(axis=0).edges()[:-1]
            yaxis = myFile[key].axis(axis=1).edges()[:-1]

In [ ]:
# Convert TOF channels to microseconds:
# a TOF channel width is 2773 ticks and one tick is 25ns.
# Then also add a 6.25ms delay
xaxis = (xaxis * TOF_Time_Channel_Width + TOF_Window_Delay_Register) * tof_tick

In [ ]:
fig9, ax9 = plt.subplots()
cont9 = ax9.contourf(xaxis, yaxis, data2d_root_sp11, cmap=colormap)
ax9.set_title(ROOT_file_sp11)
cbar9 = fig9.colorbar(cont9)

In [ ]:
# Note I had to change the distance to the detector again from 30.5 -> 30.0
v20setup["info"]["detector_position"] = 30.0
# Get WFM frame parameters and generate TOF diagram
frames = wfm.get_frames(instrument=v20setup, plot=True)

In [ ]:
# Plot frame locations onto raw data
fig10, ax10 = plt.subplots()
ax10.plot(xaxis, np.sum(data2d_root_sp11, axis=0))
ax10.grid()
ax10.set_title('ROOT Spectrum11 - frame boundaries for stitching')
for gap in frames['DENEX']['gaps']:
    ax10.axvline(x=gap, linestyle='--', color='r');

In [ ]:
# Stitch the 2d histogram
root_stitched_sp11 = wfm.stitch(x=xaxis, y=data2d_root_sp11, frames=frames['DENEX'])

In [ ]:
fig11, ax11 = plt.subplots(2, 1, constrained_layout=True, sharex=True)
ax11[0].set_title('Final stitched plot - ROOT Spectrum 11')
cont11 = ax11[0].contourf(xaxis, yaxis, root_stitched_sp11, cmap=colormap)
fig11.colorbar(cont11, ax=[ax11[0]], location='right')
ax11[1].plot(xaxis, np.sum(root_stitched_sp11, axis=0))
ax11[1].grid()

In [ ]:
fig12, ax12 = plt.subplots(1, 2, constrained_layout=True)
fig12.suptitle('ROOT Spectrum 11')
ax12[0].contourf(xaxis, yaxis, data2d_root_sp11, cmap=colormap)
ax12[0].set_title('Raw data')

ax12[1].set_title('Stitched data')
ax12[1].contourf(xaxis, yaxis, root_stitched_sp11, cmap=colormap)

## Spectrum 12

In [ ]:
# open a ROOT file and extract only one 2D dataset specified in selected_dataset
# Note the vertical axis of 2D datasets is inverted

ROOT_file_sp12 = 'Spectrum12_DENEX006_1_18-02-10_0000.root'
path_to_root_file = os.path.join(dataconfig.data_root, ROOT_file_sp12)
key_spectrum ='Spectrum12'
dir_with_data = 'Meas_1'
selected_dataset = 'H_TOF,X1-X2_User_2D4_dsp_after_run_1'

with uproot.open(path_to_root_file)[dir_with_data] as myFile:

    for key in myFile.keys():
        if 'BoardParam_run' in str(key):
            myObject = myFile[key]
            nb_xbins = int(myObject.member('fEntries'))
                        
            for i in range(nb_xbins):
                if 'TOF_Time_Channel_Width' in myObject.axis(axis=0).labels()[i]:
                    TOF_Time_Channel_Width = myObject.counts(False)[i]
                elif 'TOF_Window_Delay_Register' in myObject.axis(axis=0).labels()[i]:
                    TOF_Window_Delay_Register = myObject.counts(False)[i]  
                # print(f"{myObject.axis(axis=0).labels()[i]}: {myObject.counts(False)[i]}")

        # 2D contourplot
        if selected_dataset in str(key):
            data2d_root_sp12 =  np.flip(myFile[key].counts(False), 1).transpose()
            # create x- and y-axis
            xaxis = myFile[key].axis(axis=0).edges()[:-1]
            yaxis = myFile[key].axis(axis=1).edges()[:-1]

In [ ]:
# Convert TOF channels to microseconds:
# a TOF channel width is 2773 ticks and one tick is 25ns.
# Then also add a 6.25ms delay
xaxis = (xaxis * TOF_Time_Channel_Width + TOF_Window_Delay_Register) * tof_tick

In [ ]:
fig13, ax13 = plt.subplots()
cont13 = ax13.contourf(xaxis, yaxis, data2d_root_sp12, cmap=colormap)
ax13.set_title(ROOT_file_sp12)
cbar13 = fig13.colorbar(cont13)

In [ ]:
# Note I had to change the distance to the detector again from 30.5 -> 30.0
v20setup["info"]["detector_position"] = 30.0
# Get WFM frame parameters and generate TOF diagram
frames = wfm.get_frames(instrument=v20setup, plot=True)

In [ ]:
# Plot frame locations onto raw data
fig14, ax14 = plt.subplots()
ax14.plot(xaxis, np.sum(data2d_root_sp12, axis=0))
ax14.grid()
ax14.set_title('ROOT - frame boundaries for stitching')
for gap in frames['DENEX']['gaps']:
    ax14.axvline(x=gap, linestyle='--', color='r')

In [ ]:
# Stitch the 2d histogram
root_stitched_sp12 = wfm.stitch(x=xaxis, 
                                y=data2d_root_sp12, 
                                frames=frames['DENEX'])

In [ ]:
fig15, ax15 = plt.subplots(2, 1, constrained_layout=True, sharex=True)
ax15[0].set_title('Final stitched plot - ROOT Spectrum 12')
cont15 = ax15[0].contourf(xaxis, yaxis, root_stitched_sp12, cmap=colormap)
fig15.colorbar(cont15, ax=[ax15[0]], location='right')
ax15[1].plot(xaxis, np.sum(root_stitched_sp12, axis=0))
ax15[1].grid()

In [ ]:
fig16, ax16 = plt.subplots(1, 2, constrained_layout=True)
fig16.suptitle('ROOT Spectrum 12')
ax16[0].contourf(xaxis, yaxis, data2d_root_sp12, cmap=colormap)
ax16[0].set_title('Raw data')

ax16[1].set_title('Stitched data')
ax16[1].contourf(xaxis, yaxis, root_stitched_sp12, cmap=colormap);